# this script allows to print all the information for a specific SNP

In [1]:
import pandas as pd
import numpy as np
from multiprocessing import Pool
from functools import partial
import glob
import os
import plotly.express as px
import math
import matplotlib.pyplot as plt
from matplotlib import colors
import pybedtools as pbed

from scipy import stats, special
from statsmodels.stats import multitest
import statsmodels.api as sm
import statsmodels.formula.api as smf
import plotly.io as pio
import seaborn as sns

from functools import reduce

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_columns', 500)

base_dir = "http://bartzabel.ls.manchester.ac.uk/orozcolab/SNP2Mechanism/"


In [2]:
gtf_annotation_df = pd.read_pickle(f"{base_dir}/metadata/gencode_gtf.pickle")
gtf_transcripts = gtf_annotation_df[(gtf_annotation_df["feature"] == "transcript") & (gtf_annotation_df["transcript_type"] == "protein_coding")].dropna(axis=1, how='all')
gtf_transcripts["gene_id"] = gtf_transcripts["gene_id"].str.split(".").str[0]
gtf_transcripts["transcript_id"] = gtf_transcripts["transcript_id"].str.split(".").str[0]
gtf_transcripts["TSS_start"] = gtf_transcripts.apply(lambda x: int(x["start"]) if x["strand"] == "+" else int(x["end"]) ,axis = 1)
gene_mapper = dict(zip(gtf_transcripts['gene_id'], gtf_transcripts['gene_name']))

In [3]:
RNA_nominal_CD4 = pd.read_csv(f"{base_dir}/QTLs/RNA/RNA_nominal_CD4_merged.txt", sep = " ")
RNA_nominal_CD8 = pd.read_csv(f"{base_dir}/QTLs/RNA/RNA_nominal_CD8_merged.txt", sep = " ")
ATAC_nominal_CD8 = pd.read_csv(f"{base_dir}/QTLs/ATAC/ATAC_nominal_CD8_merged.txt", sep = " ")
ATAC_nominal_CD4 = pd.read_csv(f"{base_dir}/QTLs/ATAC/ATAC_nominal_CD4_merged.txt", sep = " ")
ins_nominal_CD8 = pd.read_csv(f"{base_dir}/QTLs/HiC/ins_nominal_CD8_merged.txt", sep = " ")
ins_nominal_CD4 = pd.read_csv(f"{base_dir}/QTLs/HiC/ins_nominal_CD4_merged.txt", sep = " ")
loop_nominal_CD8 = pd.read_csv(f"{base_dir}/QTLs/HiC/loop_nominal_CD8_merged.txt", sep = " ")
loop_nominal_CD4 = pd.read_csv(f"{base_dir}/QTLs/HiC/loop_nominal_CD4_merged.txt", sep = " ")


RNA_permuted_CD4 = pd.read_csv(f"{base_dir}/QTLs/RNA/RNA_permuted_CD4_FDR.txt", sep = " ")
RNA_permuted_CD8 = pd.read_csv(f"{base_dir}/QTLs/RNA/RNA_permuted_CD8_FDR.txt", sep = " ")
ATAC_permuted_CD4 = pd.read_csv(f"{base_dir}/QTLs/ATAC/ATAC_permuted_CD4_FDR.txt", sep = " ")
ATAC_permuted_CD8 = pd.read_csv(f"{base_dir}/QTLs/ATAC/ATAC_permuted_CD8_FDR.txt", sep = " ")
ins_permuted_CD4 = pd.read_csv(f"{base_dir}/QTLs/HiC/ins_permuted_CD4_FDR.txt", sep = " ")
ins_permuted_CD8 = pd.read_csv(f"{base_dir}/QTLs/HiC/ins_permuted_CD8_FDR.txt", sep = " ")
loop_permuted_CD4 = pd.read_csv(f"{base_dir}/QTLs/HiC/loop_permuted_CD4_FDR.txt", sep = " ")
loop_permuted_CD8 = pd.read_csv(f"{base_dir}/QTLs/HiC/loop_permuted_CD8_FDR.txt", sep = " ")

loop_allelic_imbalance_ALL = pd.read_csv(f"../HiC_allelic_imbalance/.local/results/allelic_imbalance_ALL_apeglm_results.csv")
loop_allelic_imbalance_CD8 = pd.read_csv(f"../HiC_allelic_imbalance/.local/results/allelic_imbalance_CD8_apeglm_results.csv")
loop_allelic_imbalance_CD4 = pd.read_csv(f"../HiC_allelic_imbalance/.local/results/allelic_imbalance_CD4_apeglm_results.csv")

atac_allelic_imbalance_all = pd.read_csv("../ATAC_allelic_imbalance/.local/results/ATAC_ALL_allelic_imbalance_with_betabinom.csv.gz", index_col = 0)
atac_allelic_imbalance_CD8 = pd.read_csv("../ATAC_allelic_imbalance/.local/results/ATAC_CD8_allelic_imbalance_with_betabinom.csv.gz", index_col = 0)
atac_allelic_imbalance_CD4 = pd.read_csv("../ATAC_allelic_imbalance/.local/results/ATAC_CD4_allelic_imbalance_with_betabinom.csv.gz", index_col = 0)


In [4]:
RNA_nominal_CD4["gene_name"] = RNA_nominal_CD4['phe_id'].map(gene_mapper)
RNA_nominal_CD8["gene_name"] = RNA_nominal_CD8['phe_id'].map(gene_mapper)

In [5]:
def identify_QTL_permuted(var, nominal, permuted):
    # get list of all QTLs that are from the region of the nominal
    phenotypes_with_nominal_qtl = nominal[nominal["var_id"].isin(var)]["phe_id"].to_list()
    display(nominal[nominal["var_id"].isin(var)])
    display(permuted[permuted["phe_id"].isin(phenotypes_with_nominal_qtl)].drop(columns = ["dof1","dof2","bml1","bml2"]))

In [6]:
def get_all_tables(vars):
    if not type(vars) == list:
        vars = [vars]
    print("eQTL CD4")
    identify_QTL_permuted(vars, RNA_nominal_CD4, RNA_permuted_CD4)

    print("eQTL CD8")
    identify_QTL_permuted(vars, RNA_nominal_CD8, RNA_permuted_CD8)

    print("caQTL CD4")
    identify_QTL_permuted(vars, ATAC_nominal_CD4, ATAC_permuted_CD4)

    print("caQTL CD8")
    identify_QTL_permuted(vars, ATAC_nominal_CD8, ATAC_permuted_CD8)

    print("loopQTL CD4")
    identify_QTL_permuted(vars, loop_nominal_CD4, loop_permuted_CD4)

    print("loopQTL CD8")
    identify_QTL_permuted(vars, loop_nominal_CD8, loop_permuted_CD8)

    print("insQTL CD4")
    identify_QTL_permuted(vars, ins_nominal_CD4, ins_permuted_CD4)

    print("insQTL CD8")
    identify_QTL_permuted(vars, ins_nominal_CD8, ins_permuted_CD8)


    print("allelic imbalance ALL")
    display(atac_allelic_imbalance_all[atac_allelic_imbalance_all["ID"].isin(vars)])

    print("allelic imbalance CD4")
    display(atac_allelic_imbalance_CD4[atac_allelic_imbalance_CD4["ID"].isin(vars)])

    print("allelic imbalance CD8")
    display(atac_allelic_imbalance_CD8[atac_allelic_imbalance_CD8["ID"].isin(vars)])


    print("loops with allelic imbalance ALL")
    display(loop_allelic_imbalance_ALL[loop_allelic_imbalance_ALL["rsID"].isin(vars)])

    print("loops with allelic imbalance CD4")
    display(loop_allelic_imbalance_CD4[loop_allelic_imbalance_CD4["rsID"].isin(vars)])

    print("loops with allelic imbalance CD8")
    display(loop_allelic_imbalance_CD8[loop_allelic_imbalance_CD8["rsID"].isin(vars)])


In [26]:
get_all_tables("rs13401811")

eQTL CD4


,phe_id,phe_chr,phe_from,phe_to,phe_strd,n_var_in_cis,dist_phe_var,var_id,var_chr,var_from,var_to,nom_pval,r_squared,slope,best_hit,gene_name


,phe_id,phe_chr,phe_from,phe_to,phe_strd,n_var_in_cis,dist_phe_var,var_id,var_chr,var_from,var_to,nom_pval,r_squared,slope,adj_emp_pval,adj_beta_pval,FDR


eQTL CD8


,phe_id,phe_chr,phe_from,phe_to,phe_strd,n_var_in_cis,dist_phe_var,var_id,var_chr,var_from,var_to,nom_pval,r_squared,slope,best_hit,gene_name


...

,Unnamed: 0,chrA,startA,endA,chrB,startB,endB,loopID,loopScore,rsID,rsCoord,REF_counts,ALT_counts,svalue


loops with allelic imbalance CD4


,Unnamed: 0,chrA,startA,endA,chrB,startB,endB,loopID,loopScore,rsID,rsCoord,REF_counts,ALT_counts,svalue


loops with allelic imbalance CD8


,Unnamed: 0,chrA,startA,endA,chrB,startB,endB,loopID,loopScore,rsID,rsCoord,REF_counts,ALT_counts,svalue
